In [1]:
from transformers import pipeline

/Users/t.taniguchi/Library/Caches/pypoetry/virtualenvs/study-lmm-wQFh0UWk-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPTを使ってみる

In [2]:
# 後続するテキストを予測するpipelineを作成
generator = pipeline(
    "text-generation", model="abeja/gpt2-large-japanese"
)
# "日本で一番高い山は"に続くテキストを生成
outputs = generator("日本で一番高い山は")
print(outputs[0]["generated_text"])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


日本で一番高い山は山頂の三角点(30,062m)から 30.5キロの所にある 焼山(やけやま) だそうで 今回は 焼山より上の 大山は


## BERTを使ってみる

In [3]:
import pandas as pd

# マスクされたトークンを予測するpipelineを作成
fill_mask = pipeline(
    "fill-mask", model="cl-tohoku/bert-base-japanese-v3"
)
masked_text = "日本の首都は[MASK]である"
# [MASK]部分を予測
outputs = fill_mask(masked_text)
# 上位3件をテーブルで表示
display(pd.DataFrame(outputs[:3]))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTra

,score,token,token_str,sequence
0,0.884172,12569,東京,日本 の 首都 は 東京 で ある
1,0.024820,12759,大阪,日本 の 首都 は 大阪 で ある
2,0.020863,13017,京都,日本 の 首都 は 京都 で ある


In [4]:
masked_text = "今日の映画は刺激的で面白かった。この映画は[MASK]。"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:3]))

,score,token,token_str,sequence
0,0.683933,23845,素晴らしい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 素晴らしい 。
1,0.101235,24683,面白い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 面白い 。
2,0.048003,26840,楽しい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 楽しい 。


## T5を使ってみる

In [5]:
# text-to-textで生成するpipelineを作成
t2t_generator = pipeline(
    "text2text-generation", model="retrieva-jp/t5-large-long"
)
# マスクされたスパンを予測
masked_text = "江戸幕府を開いたのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

Device set to use cpu


徳川家康


In [6]:
t2t_generator.tokenizer.convert_tokens_to_ids("<extra_id_1")

2

In [7]:
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

日本銀行


In [8]:
"日本銀行" in t2t_generator.tokenizer.vocab

False

"日本銀行"はモデル学習時の語彙に含まれていないので難しい。

## トークナイゼーション

In [12]:
word_freqs = {
    "たのしい": 6,
    "たのしさ": 2,
    "うつくしい": 4,
    "うつくしさ": 1,
}
# 語彙を文字で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))
# 単語とその分割状態
splits = {word: [char for char in word] for word in word_freqs}


In [13]:
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
    ) -> tuple[str, str]:
    """
    最も頻度の高い隣接するサブワードの組を計算する
    """
    pair_freqs = Counter()  # サブワードの組のカウンタ
    for word, freq in word_freqs.items():  # すべての単語を処理
        split = splits[word]  # 現在の単語の分割状態を取得
        # すべての隣接したサブワードの組を処理
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            # サブワードの組の頻度に単語の頻度を加算
            pair_freqs[pair] += freq
    # カウンタから最も頻度の高いサブワードの組を取得
    pair, _ = pair_freqs.most_common(1)[0]
    return pair

def merge_pair(
    target_pair: tuple[str, str], splits: dict[str, list[str]]
) -> dict[str, list[str]]:
    """
    サブワードの組を結合する
    """
    l_str, r_str = target_pair

    for word in word_freqs:  # すべての単語を処理
        split = splits[word]  # 現在の単語の分割状態を取得
        i = 0
        # すべての隣接したサブワードの組を処理
        while i < len(split) - 1:
            # サブワードの組が結合対象と一致したら結合
            if split[i] == l_str and split[i + 1] == r_str:
                split = split[:i] + [l_str + r_str] + split[i + 2 :]
            i += 1
        splits[word] = split  # 現在の結合状態を更新
    return splits

In [14]:
for step in range(9):
    # 最も頻度の高い隣接するサブワードの組を計算
    target_pair = compute_most_frequent_pair(splits)
    # サブワードの組を結合
    splits = merge_pair(target_pair, splits)
    # 語彙にサブワードの組を追加
    vocab.append(target_pair)

In [16]:
print(vocab)

['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]


## トークナイゼーション

In [17]:
from transformers import AutoTokenizer
bert_ja_tokenizer = AutoTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v3"
    )
print(
    bert_ja_tokenizer.tokenize("自然言語処理にディープラーニングを使う")
)

['自然', '言語', '処理', 'に', 'ディープ', 'ラー', '##ニング', 'を', '使う']
